# Demultiplexing ERA Raw Sequence Files for DADA2 Pipeline

### Set up workspace and read in mapping file.

* Mapping file could contain 3 columns:
    * Sample Name
    * Index 1 barcode 
    * Index 2 barcode

In [1]:
import pandas as pd
import screed
from itertools import izip
from collections import Counter
import gzip
import matplotlib.pyplot as plt
#import seaborn as sns
import pickle

from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models.glyphs import Circle
from bokeh.models import ColumnDataSource
from bokeh.models import HoverTool, BoxZoomTool, ResetTool, CustomJS, CrosshairTool
output_notebook()

%matplotlib inline

Loading BokehJS ...

In [2]:
    ir_df = pd.read_table("/home/bryan/ERA/ERA_MappingFile.txt")

In [3]:
ir_df.head()

,X.sampleID,TubeID,ExtractionPlate,PooledDNAPlate,Sample Well ID,PrimerPlate,Primer Number,Primer Well ID,Unnamed: 8,primerFwdLoc,primerRevLoc,fwd_barcode,rev_barcode,PlotID,TimePoint
0,ERA-T1_1-2b_170,125,ERA1,A,A1,1,1,A1,NaN,1-A,1-1,ATCGTACG,AACTCTCG,1-2b_170,T1
1,ERA-T1_1-3c_170,137,ERA1,A,B1,1,2,B1,NaN,1-B,1-1,ACTATCTG,AACTCTCG,1-3c_170,T1
2,ERA-T3_1-2b_0,530,ERA1,A,C1,1,3,C1,NaN,1-C,1-1,TAGCGAGT,AACTCTCG,1-2b_0,T3
3,ERA-T2_1-2b_85,361,ERA1,A,D1,1,4,D1,NaN,1-D,1-1,CTGCGTGT,AACTCTCG,1-2b_85,T2
4,ERA-T3_1-2d_170,602,ERA1,A,E1,1,5,E1,NaN,1-E,1-1,TCATCGAG,AACTCTCG,1-2d_170,T3


In [4]:
# rev-comp functions
def complement(seq): 
    idx = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A', 'N':'N'} 
    idx.update( {x.lower():y.lower() for x,y in idx.items()} )
    return ''.join(idx[x] for x in list(seq))

def revComp(seq):
    seq = seq[::-1]
    return complement(seq) 

In [5]:
Barcode_R = [(revComp(y)).upper() for y \
                         in ir_df.rev_barcode]
ir_df.rev_barcode = Barcode_R
ir_df.head()

,X.sampleID,TubeID,ExtractionPlate,PooledDNAPlate,Sample Well ID,PrimerPlate,Primer Number,Primer Well ID,Unnamed: 8,primerFwdLoc,primerRevLoc,fwd_barcode,rev_barcode,PlotID,TimePoint
0,ERA-T1_1-2b_170,125,ERA1,A,A1,1,1,A1,NaN,1-A,1-1,ATCGTACG,CGAGAGTT,1-2b_170,T1
1,ERA-T1_1-3c_170,137,ERA1,A,B1,1,2,B1,NaN,1-B,1-1,ACTATCTG,CGAGAGTT,1-3c_170,T1
2,ERA-T3_1-2b_0,530,ERA1,A,C1,1,3,C1,NaN,1-C,1-1,TAGCGAGT,CGAGAGTT,1-2b_0,T3
3,ERA-T2_1-2b_85,361,ERA1,A,D1,1,4,D1,NaN,1-D,1-1,CTGCGTGT,CGAGAGTT,1-2b_85,T2
4,ERA-T3_1-2d_170,602,ERA1,A,E1,1,5,E1,NaN,1-E,1-1,TCATCGAG,CGAGAGTT,1-2d_170,T3


## Check that sequences are in the same order between read and index files

Sequence names begin with the '@' symbol in each file. Check that the characters following the '@' symbol are identical between each file

In [6]:
!zcat /home/bryan/ERA/data/MiSeq/20170417_run1/index1.fq.gz  | head

@M02465:355:000000000-B3LCK:1:1101:15651:1332 1:N:0:0
TAGTCTCC
+
CCCBCFFF
@M02465:355:000000000-B3LCK:1:1101:15673:1332 1:N:0:0
TAGTCTCC
+
>A1>>3D1
@M02465:355:000000000-B3LCK:1:1101:15902:1335 1:N:0:0
GTCTATGA

gzip: stdout: Broken pipe


In [7]:
!zcat /home/bryan/ERA/data/MiSeq/20170417_run1/read1.fq.gz | head

@M02465:355:000000000-B3LCK:1:1101:15651:1332 1:N:0:0
TACGTAGGGGCCGAGCGTTGTCCGGAGTTACTGGGCGTAAAGCGCGCGCAGGCGGCTCGCTTTGCCCGGCGTGAAAGCCCCCGGCTCAACCGGGGAGGGTCGTCGGGGACGGGCGAGCTTGAGGCCGGCAGGGGCAGGTGGAATTCCCGGTGTAGTGGTGAAATGCGTAGAGATCGGGAGGAACACCCGTGGCGAAGGCGGCCTGCTGGGCCGGACCTGACGCTGAGGCGCGAAGGCGTGGGGGGCGAAAG
+
AABA>C4AB@DDGCECGEFEGGHFCD2EHGFHHGDGEGGFHHHGGGGGGE?@EEEGGGG/?//FGHCGCGGGGGGGHHHHGGGGDADFEFGCGGGCGGG;CBGGGGGG@?-><@-B?BDFFEFF?EBBBBBBBBB-AB9BE./BBBBFF?=BD9FBBF9E/;9FFFDD.B?FFFFFFBBBB.B9BBDDBAD?=B=9BD??BB99..;FFDFA?BBB-.9BBA?9DDFFFBB=--9:A?99;B<-;-;---.
@M02465:355:000000000-B3LCK:1:1101:15673:1332 1:N:0:0
TACGGAGGGTGCAAGCGTTATCCGGATTCACTGGGTTTAAAGGGTGCGTATGTGGGCAGGTAAGTCAGTGGTGAAATCCCCGAGCTTAACTTGGGAACTGCCGTTGATACTGTCTGTCTTGAGTCTCGTGGAGGTTGGCGGAATTTGTCATGTAGCGGTGAAATGCGTAGAGTTGACATGGAACACCATTTGCGAAGGCAGCTGGCTAGACGGTACTTACCTCTGTGGCACGAAGGCGTGGGGAGCAAACG
+
AAAA1>A1>A1A1BBFEGG0A3FE?EEG11ADFFC0/EFGG1AE0BAEFA/B22B/F0/>GD2FEGHFHHFGFF1EFHHHEE/E>EFF1G2BBGE/F/GHHF?EEEHHG>B2BB21FFHHFB1F111BFF//

### Create new R1 and R2 files for each sample based on index barcodes

This block will read in your read1, read2, index1, and index2 files, compare barcode sequences to the provided mapping file, and create new files based on the sample names provided in the mapping file.

In [8]:
read1_fhs = dict([(s, gzip.open("/home/bryan/ERA/data/MiSeq/20170417_run1/v4bac_lib1_demult/%s.R1.fq.gz"%s, "w")) for s in ir_df["X.sampleID"]])
read2_fhs = dict([(s, gzip.open("/home/bryan/ERA/data/MiSeq/20170417_run1/v4bac_lib1_demult/%s.R2.fq.gz"%s, "w")) for s in ir_df["X.sampleID"]])

idx_d = dict([(row["rev_barcode"].upper() + row["fwd_barcode"].upper(), row["X.sampleID"]) for i, row in ir_df.iterrows()])

path = "/home/bryan/ERA/data/MiSeq/20170417_run1/"
ir1 = screed.open(path + "index1.fq.gz")
ir2 = screed.open(path + "index2.fq.gz")
r1 = screed.open(path + "read1.fq.gz")
r2 = screed.open(path + "read2.fq.gz")

seq_cnt = Counter()
bc_cnt = Counter()
bad_bc_count = 0

for i, recs in enumerate(izip(ir1, ir2, r1, r2)):
    ir1 = recs[0]
    ir2 = recs[1]
    r1 = recs[2]
    r2 = recs[3]
        
    idx_key = ir1.sequence + ir2.sequence
    bc_cnt[idx_key] += 1
    
    try:
        sample_name = idx_d[idx_key]
    except KeyError:
        bad_bc_count += 1
        continue
    
    seq_cnt[sample_name] += 1
    
    fastq1 = "@%s\n%s\n+\n%s\n"%(r1.name, r1.sequence, r1.quality)
    read1_fhs[sample_name].write(fastq1)

    fastq2 = "@%s\n%s\n+\n%s\n"%(r2.name, r2.sequence, r2.quality)
    read2_fhs[sample_name].write(fastq2)
    
for i, j in zip(read1_fhs.itervalues(), read2_fhs.itervalues()):
    i.close(), j.close()

# not edited below here

In [9]:
print "We found %s barcodes that did not match the mapping file."%str(bad_bc_count)

We found 3207757 barcodes that did not match the mapping file.


In [10]:
pickle.dump(seq_cnt, open("/home/bryan/ERA/data/MiSeq/20170417_run1/v4bac_lib1_demult/seq_cnt.pickle", "w"))
pickle.dump(bc_cnt, open("/home/bryan/ERA/data/MiSeq/20170417_run1/v4bac_lib1_demult/bc_cnt.pickle", "w"))

In [11]:
seq_cnt = pickle.load(open("/home/bryan/ERA/data/MiSeq/20170417_run1/v4bac_lib1_demult/seq_cnt.pickle"))

### Check Sequence counts per samples

At this point, the demultiplexing is complete, but you might want to check the total number of sequences obtained for each sample.

In [12]:
df = pd.DataFrame.from_dict(dict(seq_cnt.most_common()), orient="index")
df.rename(columns={0 : "count"}, inplace=True)
df.sort(columns="count", inplace=True, ascending=False)
df["color"] = "#F8766D"

df.reset_index(inplace=True)
df.rename(columns={"index" : "SampleID"}, inplace=True)
df["x"] = [i + 1 for i, s in enumerate(df["SampleID"])]

p = figure(width=800, height=400, y_axis_type = "log", 
           tools="", toolbar_location="left", 
           y_axis_label = "Seq count", x_axis_label = "Sample")

p.xaxis.axis_line_width = 3
p.yaxis.axis_line_width = 3
p.outline_line_color = None
p.grid.grid_line_color = None

source = ColumnDataSource(df)

invisible_circle = Circle(x='x', y='count', 
                          fill_color='color', 
                          fill_alpha=0.5, 
                          line_color="color", 
                          line_alpha = 0.5, size=8)


visible_circle = Circle(x='x', y='count', 
                        fill_color='color', 
                        fill_alpha=1.0, 
                        line_color="color")

cr = p.add_glyph(source, 
                 invisible_circle, 
                 selection_glyph=visible_circle, 
                 nonselection_glyph=invisible_circle)

l = p.line(x = df["x"], 
           y = df["count"], 
           line_width=3, 
           color='#F8766D')

code = "source.set('selected', cb_data['index']);"
callback = CustomJS(args={'source': source}, code=code)

tooltip = """
    <div>
        <span style="font-size: 17px; font-weight: bold;">@SampleID </span>
    </div>
    <div>
        <span style="font-size: 17px; font-weight: bold;">@count </span>
    </div>
"""

p.add_tools(HoverTool(tooltips=tooltip, callback=callback, renderers=[cr]),
            BoxZoomTool(dimensions=["width"]),
            ResetTool(),
            CrosshairTool(dimensions = ["height"]))

p.xaxis.major_label_text_color = "white"

sh = show(p), 

/opt/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  app.launch_new_instance()
/opt/anaconda2/lib/python2.7/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: List(Enum(Dimension)) in BoxZoomTool.dimensions was deprecated in Bokeh 0.12.3 and will be removed, use Enum(Dimensions) instead.
  warn(message)
/opt/anaconda2/lib/python2.7/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: List(Enum(Dimension)) in CrosshairTool.dimensions was deprecated in Bokeh 0.12.3 and will be removed, use Enum(Dimensions) instead.
  warn(message)


In [13]:
seq_cnt.most_common(248)

[('ERA-T0_4-3d_0', 186162),
 ('ERA-T3_2-5a_170', 135144),
 ('MockComm', 118623),
 ('ERA-T3_2-2b_0', 117364),
 ('ERA-T1_3-2c_85', 116568),
 ('ERA-T3_2-3b_0', 114862),
 ('ERA-T0_1-2d_0', 113615),
 ('ERA-T1_2-5c_0', 110838),
 ('ERA-T2_2-4a_85', 109993),
 ('ERA-T2_2-2d_85', 108370),
 ('ERA-T2_4-3c_85', 102244),
 ('ERA-T0_1-5a_0', 102130),
 ('ERA-T3_4-3d_170', 99900),
 ('ERA-T0_2-2c_0', 98769),
 ('ERA-T1_1-5a_170', 98744),
 ('ERA-T3_3-2c_170', 97371),
 ('ERA-T2_1-5a_85', 96768),
 ('ERA-T3_2-2b_85', 96487),
 ('ERA-T3_2-5a_0', 95367),
 ('ERA-T1_2-3d_85', 95101),
 ('ERA-T1_2-3b_0', 94463),
 ('ERA-T3_2-2d_170', 93485),
 ('ERA-T3_4-3d_0', 92645),
 ('ERA-T1_4-3d_170', 91543),
 ('ERA-T1_2-3b_85', 91537),
 ('ERA-T3_2-5c_170', 90226),
 ('ERA-T2_4-3d_85', 90057),
 ('ERA-T2_4-5c_85', 89873),
 ('ERA-T3_4-3d_85', 88622),
 ('ERA-T3_4-3b_170', 87887),
 ('ERA-T1_3-2c_170', 84477),
 ('ERA-T1_2-3d_0', 82052),
 ('ERA-T1_4-3d_85', 80820),
 ('ERA-T3_2-2d_0', 78377),
 ('ERA-T3_4-4a_85', 77916),
 ('ERA-T3_1-4a_0'

In [14]:
df[df["count"] < 200000].shape

(253, 4)